# # Chord Progressions Notebook
En este notebook:

1. Generamos archivos .mid con `generate_progression.py` en TODAS las tonalidades (p. ej. "A-1-..." o "C#-5-...").
2. Convertimos esos .mid a .wav con Timidity usando `audio_conversion.py`.
3. Creamos .jams parseando el nombre de cada .mid para extraer:
   - Nota y octava (p. ej. "A-1"),
   - Progresión en numerales romanos (p. ej. "I-vi-IV-V"),
   - Índice final, p. ej. "3".

---

## 1. Imports y Configuración

In [2]:
import sys
from pathlib import Path

# Ajustar la ruta para que Python reconozca 'src'
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR, DEFAULT_TEMPO
from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_file

print("MIDI_DIR =", MIDI_DIR)
print("WAV_DIR  =", WAV_DIR)
print("JAMS_DIR =", JAMS_DIR)
print("DEFAULT_TEMPO =", DEFAULT_TEMPO)



MIDI_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi
WAV_DIR  = /home/cepatinog/MIR/final_project/chord_progression_generator/data2/wav
JAMS_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data2/jams
DEFAULT_TEMPO = 60


## 2. Generar Progresiones .mid


Cada una se expandirá en las diferentes octavas y tonalidades, generando múltiples .mid en `data/midi/...`.


In [ ]:
# Todos los tipos de acordes posibles
prog_all_1 = "i,min-ii,maj-iii,dim-IV,aug"
prog_all_2 = "V,min6-vi,maj6-vii,min7-I,minmaj7"
prog_all_3 = "ii,maj7-iii,7-IV,dim7-V,hdim7"
prog_all_4 = "vi,sus2-vii,sus4-I,maj-vi,min"

name_all_1 = "i,min-ii,maj-iii,dim-IV,aug"
name_all_2 = "V,min6-vi,maj6-vii,min7-I,minmaj7"
name_all_3 = "ii,maj7-iii,7-IV,dim7-V,hdim7"
name_all_4 = "vi,sus2-vii,sus4-I,maj-vi,min"

generate_progression(prog_all_1, name_all_1, MIDI_DIR, DEFAULT_TEMPO)
generate_progression(prog_all_2, name_all_2, MIDI_DIR, DEFAULT_TEMPO)
generate_progression(prog_all_3, name_all_3, MIDI_DIR, DEFAULT_TEMPO)
generate_progression(prog_all_4, name_all_4, MIDI_DIR, DEFAULT_TEMPO)

# Rutas de carpetas generadas
midi_folder_all_1 = MIDI_DIR / name_all_1
midi_folder_all_2 = MIDI_DIR / name_all_2
midi_folder_all_3 = MIDI_DIR / name_all_3
midi_folder_all_4 = MIDI_DIR / name_all_4




## 3. Crear .jams parseando el nombre de cada .mid y limpiando subguiones
No asumimos una sola tonalidad. En su lugar:
- extraemos `<noteName>-<oct>` como tonalidad (ej: `Eb-6` => `Eb`),
- extraemos la progresión entre `parts[2:-1]`,
- dividimos los tokens y cada uno lo limpiamos con `strip_suffix` para quitar `_algo`.



In [ ]:
def strip_suffix(token: str) -> str:
    if "_" in token:
        return token.split("_", 1)[0]
    return token

def parse_filename_and_create_jams(midi_file: Path):
    base_name = midi_file.stem
    parts = base_name.split("-")
    if len(parts) < 3:
        print(f"Nombre de archivo inesperado => {base_name}")
        return

    note_plus_oct = "-".join(parts[:2])
    progression_list = parts[2:-1]
    raw_tokens = "-".join(progression_list).split("-")
    roman_sequence = [strip_suffix(tok) for tok in raw_tokens]

    splitted = note_plus_oct.split("-")
    tonalidad = strip_suffix(splitted[0]) if len(splitted) >= 1 else "C"

    jam_path = create_jams_file(
        roman_sequence=roman_sequence,
        key=tonalidad,
        jam_name=base_name,
        progression_name="-".join(roman_sequence)
    )
    print(f"[JAMS] {midi_file.name} => {jam_path.name}")

def parse_and_create_jams_in_folder(folder: Path):
    if not folder.exists():
        print(f"No existe carpeta => {folder}")
        return
    mid_files = list(folder.rglob("*.mid"))
    if not mid_files:
        print(f"No hay .mid en => {folder}")
        return

    for mf in mid_files:
        parse_filename_and_create_jams(mf)

# Crear .jams para todas las progresiones
parse_and_create_jams_in_folder(midi_folder_all_1)
parse_and_create_jams_in_folder(midi_folder_all_2)
parse_and_create_jams_in_folder(midi_folder_all_3)
parse_and_create_jams_in_folder(midi_folder_all_4)



## 4. Verificar .jams



In [ ]:
for jfile in (JAMS_DIR / name_all_1).rglob("*.jams"):
    print("[JAMS All #1]", jfile)

for jfile in (JAMS_DIR / name_all_2).rglob("*.jams"):
    print("[JAMS All #2]", jfile)

for jfile in (JAMS_DIR / name_all_3).rglob("*.jams"):
    print("[JAMS All #3]", jfile)

for jfile in (JAMS_DIR / name_all_4).rglob("*.jams"):
    print("[JAMS All #4]", jfile)


📌 5. Convertir a WAV con Timidity

In [4]:
midi_folder_all_1 = MIDI_DIR
convert_all_mid_in_folder(midi_folder_all_1)

print("Archivos .wav creados en data/wav/")

Playing /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi/rand3_vii-III-VI_maj7_hdim7_dim/G-2-rand3_vii-III-VI_maj7_hdim7_dim-35.mid
MIDI file: /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi/rand3_vii-III-VI_maj7_hdim7_dim/G-2-rand3_vii-III-VI_maj7_hdim7_dim-35.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~6 seconds
Notes cut: 0
Notes lost totally: 0
Convertido: /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi/rand3_vii-III-VI_maj7_hdim7_dim/G-2-rand3_vii-III-VI_maj7_hdim7_dim-35.mid => /home/cepatinog/MIR/final_project/chord_progression_generator/data2/wav/G-2-rand3_vii-III-VI_maj7_hdim7_dim-35.wav
Playing /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi/rand3_vii-III-VI_maj7_hdim7_dim/G-3-rand3_vii-III-VI_maj7_hdim7_dim-30.mid
MIDI file: /home/cepatinog/MIR/final_project/chord_progression_generator/data2/midi/rand3_vii-III-VI_maj7_hdim7_dim/G-3-rand3_vii-III-VI_maj7_hdim7_dim-30

Terminated sig=0x02


KeyboardInterrupt: 